# Исследование надёжности заёмщиков
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита
# Что предстоить сделать?
- определить и заполнить пропущенные значения:
- заменить вещественный тип данных на целочисленный:
- удалить дубликаты
- выделить леммы в значениях столбца с целями получения кредита
- категоризировать данные

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — days_employed. Другой столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных имеются артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed. Заменим отрицательные значения.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Посмотрим для каждого типа занятости медианное значение трудового стажа days_employed в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Посмотрим на столбец children На предмет уникальных значений

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Имеются два аномальных значения,удалим строки с ними

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Далее заполниим пропуски в столбце days_employed медианными значениями для каждого типа занятости income_type.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [13]:
data['total_income'] = data['total_income'].astype(int)

In [14]:
data.duplicated().sum()

54

In [15]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, по уровню заработка, создадим столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [16]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [17]:
data['total_income_category'] = data['total_income'].apply(categorize_income)
display(data['total_income_category'])

0        B
1        C
2        C
3        B
4        C
        ..
21520    B
21521    C
21522    C
21523    B
21524    C
Name: total_income_category, Length: 21348, dtype: object

Перечень уникальных целей взятия кредита

In [18]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'.

In [19]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [20]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [21]:
data_children = data.groupby(['children']).agg({'debt': 'mean'})  
display(data_children)

,debt
children,
0,0.075353
1,0.092327
2,0.094542
3,0.081818
4,0.097561
5,0.000000


Сделал группу по детям и среднее по задолженности. Больше всего процент по задолженности у семей с детьми( 1 ребенок 9%, 2 ребенка 9,4%, 4 ребенка 9,7%, меньше всего с 5ю детьми.

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [22]:
data_family = data.groupby(['family_status']).agg({'debt': 'mean'}) 
display(data_family)

,debt
family_status,
Не женат / не замужем,0.097639
в разводе,0.070648
вдовец / вдова,0.066246
гражданский брак,0.092861
женат / замужем,0.075575


Получается что больше долгов у группы "не женат / не замужем" 9,7%, и меньше всего у "вдовец / вдова" - 6%

 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
data.groupby(['total_income_category'])['debt'].agg('mean')

total_income_category
A    0.080000
B    0.070602
C    0.084891
D    0.060172
E    0.090909
Name: debt, dtype: float64

Логично предположить, что чаще всего должниками становятся люди с небольшим доходом, которым финансовые проблемы не позволяют вовремя осуществлять платежи. Но глядя на наши результаты можно сделать вывод: выборка по заемщикам, как с наименьшим доходом, так и с наибольшим крайне недостаточна для формирования однозначных выводов. Можно думать, что E-категория самая низкооплачиваемая не может корректно распоряжаться деньгами и «влезает» в кредиты, но тогда неясно почему у А-категории также достаточно высокий процент задолженности. В общем вывод по группам А и Е неочевиден, мало данных.

Если же взять для анализа только группы В, С и D, то по полученным данным не наблюдаем зависимости "те, кто получает больше ЗП, лучше возвращает кредит». В целом это логично, так как материальный достаток - не единственная переменная, которая влияет на качество кредитной истории.

Как разные цели кредита влияют на его возврат в срок?

In [24]:
agg_func_math = {
    'debt': ['mean']
}

data_purpose = data.groupby(data['purpose_category']).agg(agg_func_math).round(5)

display(data_purpose)

,debt
,mean
purpose_category,
операции с автомобилем,0.09344
операции с недвижимостью,0.07253
получение образования,0.09250
проведение свадьбы,0.07874


Больше задолженности у тех кто берет кредиты на автомобиль, образование - 9,3% и 9.2% соответственно. Наименьшая задолженность при операциях с недвижимостью - 7,2%.

# Новый вывод по проведенному исследованию #

   ***Цель исследования*** - выявить факторы влияющие на погашение кредита в срок. Входный данные - статистика о платежеспособности клиентов.
    
   ***Основные гипотезы перед проведением исследования:***
   
    - Гипотеза 1: Имеется ли зависимость между количеством детей и возвратом кредита в срок
    - Гипотеза 2: Есть ли зависимость между семейным положением и возвратом кредита в срок
    - Гипотеза 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок
    - Гипотеза 4: Влияют ли разные цели кредита на его возврат в срок
    
  ***Гипотеза 1 подтверждена частично:***
   - 7%   доля задолженности по кредитам у заемщиков без детей;
   - 9%   доля задолженности по кредитам у заемщиков с 1 ребенком;
   - 9,4% доля задолженности по кредитам у заемщиков с 2 детьми;
    
        Для клиентов с 3,4 и 5 детьми необходимо больше данных, выборка не сбалансирована, классы в выборке имеют разные    размеры.
    
        ***Рекомендации:*** проведение дополнительного исследования на большей выборке данных.
    
  ***Гипотеза 2 подтверждена:***
   - Клиенты которые не состоят или не состояли в браке вероятность задолженности выше, чем у клиентов с узаконенным отношениями или в статусе "вдовец / вдова".
    
  ***Гипотеза 3 подтверждена частично:***
   - 9%  Самая высокая доля задолженностей по кредитам у категории Е с заработком менее 30000 
   - 8 % доля задолженностей по кредитам у категории С с заработком менее 50000-200000  
        
        ***Рекомендации:*** Необходимо больше данных по группам А(от 1.000.001), Е(<30.000), D( 30.000-50.001) для проведения повторного исследования и выявления более точной зависмости задолженности от заработка
        
  ***Гипотеза 4 полностью подтверждена:***
   - 9% доля задолженности по кредиту в категории автомобилем
   - 9% доля задолженности по кредиту в категории образование
   - 8% доля задолженности по кредиту в категории свадьба
   - 7% доля задолженности по кредиту в категории недвижимость
        
        ***Рекомендации:*** при оценки платежеспособности клиентов уделять внимание в 1,2 и 4 категориях.
        
        
   **Общие рекомендации**
   - В реализации скорингового алгоритма можно использовать 2 и 4 гипотезы
   - Гипотезы 1 и 4 необходимо повторно проверить после сбора более полной выборки
   